In [ ]:
!pip install -q transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 73.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from transformers import pipeline

In [ ]:
# to import files from googledrive
from google.colab import drive


drive.mount('/content/gdrive/', force_remount=True)
sample_size_dir = 'gdrive/My Drive/Sample size for NLP'

Mounted at /content/gdrive/


In [ ]:
df = pd.read_csv(sample_size_dir+'/data_jaya/41401_combined_data_1000.csv')
df.head()

# Approach one

Options:

"distilbert/distilbert-base-uncased-distilled-squad"

"deepset/roberta-base-squad2"

"google-bert/bert-large-uncased-whole-word-masking-finetuned-squad"





In [ ]:
# Load a pre-trained question answering pipeline
qa_pipeline = pipeline("question-answering", model="google-bert/bert-large-uncased-whole-word-masking-finetuned-squad")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at google-bert/bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# Question to compare with
question = "Has this patient been diagnosed with coronary atherosclerosis of the native coronary artery?"

In [ ]:
# Function to get yes/no and probabilities
def get_answer(context, question):
    result = qa_pipeline(question=question, context=context)
    yes_prob = result['score'] if result['answer'].lower() in ['yes', 'y', '1'] else 1 - result['score']
    label = 1 if result['answer'].lower() in ['yes', 'y', '1'] else 0
    return label, yes_prob

In [ ]:
# Calculate yes/no and probabilities for each row
df['answer_QA'], df['probability'] = zip(*df['TEXT'].apply(lambda x: get_answer(x, question)))

In [ ]:
df.head()

In [ ]:
df.to_csv(sample_size_dir+'/data_jaya/41401_combined_data_1000_w_QA_probs_bertlarge.csv')

# Approach two

another approach

from https://medium.com/saarthi-ai/build-a-smart-question-answering-system-with-fine-tuned-bert-b586e4cfa5f5

In [ ]:
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer


In [ ]:
#Model
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad', return_dict=False)

#Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# example

question = '''Why was the student group called "the Methodists?"'''

paragraph = ''' The movement which would become The United Methodist Church began in the mid-18th century within the Church of England.
            A small group of students, including John Wesley, Charles Wesley and George Whitefield, met on the Oxford University campus.
            They focused on Bible study, methodical study of scripture and living a holy life.
            Other students mocked them, saying they were the "Holy Club" and "the Methodists", being methodical and exceptionally detailed in their Bible study, opinions and disciplined lifestyle.
            Eventually, the so-called Methodists started individual societies or classes for members of the Church of England who wanted to live a more religious life. '''


In [ ]:
df.tail()

In [ ]:
df['length'] = df['TEXT'].apply(len)

In [ ]:
df_512 = df[df['length'] < 512].copy()

In [ ]:
df_512.head() # there are no texts less than 512

,ROW_ID_x,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,ROW_ID_y,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT,label,length


In [ ]:
# truncate text from beginning

df['TEXT_trunc'] = df['TEXT'].apply(lambda x: x[:512])

In [ ]:
df['TEXT_trunc'][995]

In [ ]:
# mimic example

question = '''Has this patient been diagnosed with coronary atherosclerosis of the native coronary artery?'''

paragraph = df['TEXT_trunc'][980]

In [ ]:
encoding = tokenizer.encode_plus(text=question,text_pair=paragraph, add_special_tokens=True) # add_special is not correct - add_special_token

In [ ]:
inputs = encoding['input_ids']  #Token embeddings
sentence_embedding = encoding['token_type_ids']  #Segment embeddings
tokens = tokenizer.convert_ids_to_tokens(inputs) #input tokens

In [ ]:
start_scores, end_scores = model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))

In [ ]:
type(start_scores)

torch.Tensor

In [ ]:
start_index = torch.argmax(start_scores)

end_index = torch.argmax(end_scores)

answer = ' '.join(tokens[start_index:end_index+1])

In [ ]:
answer

'history of present illness : 87 year old female with ao ##rti ##c ste ##nosis'

In [ ]:
corrected_answer = ''

for word in answer.split():

    #If it's a subword token
    if word[0:2] == '##':
        corrected_answer += word[2:]
    else:
        corrected_answer += ' ' + word

In [ ]:
corrected_answer

' history of present illness : 87 year old female with aortic stenosis'

Run on entire df

In [ ]:
def get_answer(paragraph):
    encoding = tokenizer.encode_plus(text=question, text_pair=paragraph, add_special_tokens=True)
    inputs = encoding['input_ids']
    sentence_embedding = encoding['token_type_ids']
    tokens = tokenizer.convert_ids_to_tokens(inputs)

    with torch.no_grad():
        start_scores, end_scores = model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))

    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores)

    answer = ' '.join(tokens[start_index:end_index+1])

    corrected_answer = ''
    for word in answer.split():
        if word[0:2] == '##':
            corrected_answer += word[2:]
        else:
            corrected_answer += ' ' + word

    return answer, corrected_answer.strip()


In [ ]:
df[['answer', 'corrected_answer']] = df['TEXT_trunc'].apply(lambda x: pd.Series(get_answer(x)))

In [ ]:
df.head()

In [ ]:
df.to_csv(sample_size_dir+'/data_jaya/41401_combined_data_1000_w_QA_probs_bertlarge_answers.csv')